# Numerical Greeks calculation

In this notebook, I'll build on the facilities provided by the `Instrument` class (that is, its ability to detect changes in its inputs and recalculate accordingly) to show how to calculate numerical Greeks when the engine doesn't provide them.

#### Setup

As usual, we import the QuantLib module and set the evaluation date:

In [1]:
from QuantLib import *

In [2]:
today = Date(8, October, 2014)
Settings.instance().evaluationDate = today

#### A somewhat exotic option

As an example, we'll use a knock-in barrier option:

In [3]:
option = BarrierOption(Barrier.UpIn,
                       120.0,   # barrier
                       0.0,     # rebate
                       PlainVanillaPayoff(Option.Call, 100.0),
                       EuropeanExercise(Date(8, January, 2015)))

For the purpose of this example, the market data are the underlying value, the risk-free rate and the volatility. We wrap them in quotes, so that the instrument will be notified of any changes...

In [4]:
u = SimpleQuote(100.0)
r = SimpleQuote(0.01)
sigma = SimpleQuote(0.20)

...and from the quotes we build the flat curves and the process that the engine requires.  As explained in a later notebook, we build the term structures so that they move with the evaluation date; this will be useful further on.

In [5]:
riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())

In [6]:
process = BlackScholesProcess(QuoteHandle(u),
                              YieldTermStructureHandle(riskFreeCurve),
                              BlackVolTermStructureHandle(volatility))

Finally, we build the engine (the library provides one based on an analytic formula) and set it to the option.

In [7]:
option.setPricingEngine(AnalyticBarrierEngine(process))

Now we can ask the option for its value...

In [8]:
print(option.NPV())

1.3657980739109867


...but we're not so lucky when it comes to Greeks:

In [9]:
print(option.delta())

RuntimeError: delta not provided

The engine doesn't provide the delta, so asking for it raises an error.

#### Numerical calculation

What does a quant have to do? We can use numerical differentiation to approximate the Greeks, as shown in the next figure: that is, we can approximate the derivative by calculating the option value for two slightly different values of the underlying and by taking the slope between the resulting points.

The relevant formulas are:

$$
\Delta = \frac{P(u_0+h)-P(u_0-h)}{2h} \; \; \; \; \; \;
\Gamma = \frac{P(u_0+h)-2P(u_0)+P(u_0-h)}{h^2}
$$

where $P(u)$ is the price of the option for a given value of the underlying $u$.

Thanks to the framework we set in place, getting the perturbed prices is easy enough. We just have to set the relevant quote to the new value and ask the option for its price again. Thus, we choose a small increment and start. First, we save the current value of the option...

In [10]:
u0 = u.value() ; h = 0.01

In [11]:
P0 = option.NPV() ; print(P0)

1.3657980739109867


...then we increase the underlying value and get the new option value...

In [12]:
u.setValue(u0+h)
P_plus = option.NPV() ; print(P_plus)

1.3688112201958083


...then we do the same after decreasing the underlying value.

In [13]:
u.setValue(u0-h)
P_minus = option.NPV() ; print(P_minus)

1.3627900998610207


Finally, we set the underlying value back to its current value.

In [14]:
u.setValue(u0)

Applying the formulas above give us the desired Greeks:

In [15]:
Delta = (P_plus - P_minus)/(2*h)
Gamma = (P_plus - 2*P0 + P_minus)/(h*h)
print(Delta)
print(Gamma)

0.3010560167393761
0.05172234855521651


The approach is usable for any Greek. We can use the two-sided formula above, or the one-sided formula below if we want to minimize the number of evaluations:

$$
\frac{\partial P}{\partial x} = \frac{P(x_0+h)-P(x_0)}{h}
$$

For instance, here we calculate Rho and Vega:

In [16]:
r0 = r.value() ; h = 0.0001
r.setValue(r0+h) ; P_plus = option.NPV()
r.setValue(r0)
Rho = (P_plus - P0)/h ; print(Rho)

6.531038494277386


In [17]:
sigma0 = sigma.value() ; h = 0.0001
sigma.setValue(sigma0+h) ; P_plus = option.NPV()
sigma.setValue(sigma0)
Vega = (P_plus - P0)/h ; print(Vega)

26.52519924198904


The approach for the Theta is a bit different, although it still relies on the fact that the option reacts to the change in the market data.  The problem is that we don't have the time to maturity available as a quote, as was the case for the other quantities.  Instead, since we set up the term structures so that they move with the evaluation date, we just have to set it to tomorrow's date to get the corresponding option value:

In [18]:
Settings.instance().evaluationDate = today+1
P1 = option.NPV()
h = 1.0/365
Theta = (P1-P0)/h ; print(Theta)

-10.770888399441302
